In [ ]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import filedialog

# === Step 1: Choose file via Tkinter ===
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename(title="📂 Select Your Thermal Dataset",
                                       filetypes=[("Excel files", "*.xlsx *.xls"), ("CSV files", "*.csv")])
print(f"✅ File selected: {file_path}")

# === Step 2: Load dataset ===
if file_path.endswith('.csv'):
    df = pd.read_csv(file_path)
else:
    df = pd.read_excel(file_path)

print("\n🔹 Columns found:", list(df.columns))

# === Step 3: Replace ranges like '1600–1800' with mean ===
def process_range(value):
    if isinstance(value, str) and ('-' in value or '–' in value):
        try:
            parts = [float(x.strip()) for x in value.replace('–', '-').split('-')]
            return np.mean(parts)
        except:
            return np.nan
    try:
        return float(value)
    except:
        return np.nan

numeric_cols = ['Density (kg/m³)', 'Thermal Conductivity (W/m-K)', 'Specific Heat (J/kg-K)', 'Max Temp (K)']

for col in numeric_cols:
    if col in df.columns:
        df[col] = df[col].apply(process_range)

# === Step 4: Fill missing numeric values ===
df.fillna(df.mean(numeric_only=True), inplace=True)

# === Step 5: Save cleaned version ===
df.to_csv("thermal_cleaned.csv", index=False)
print("✅ Cleaned dataset saved as thermal_cleaned.csv")
print(df.head())


TclError: no display name and no $DISPLAY environment variable

In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog

# === Step 1: Select cleaned file ===
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename(title="📂 Select Cleaned Thermal Dataset",
                                       filetypes=[("CSV Files", "*.csv"), ("Excel files", "*.xlsx *.xls")])
print(f"✅ File selected: {file_path}")

# === Step 2: Load data ===
if file_path.endswith('.csv'):
    df = pd.read_csv(file_path)
else:
    df = pd.read_excel(file_path)

# === Step 3: Compute TSEI ===
df['TSEI'] = df['Specific Heat (J/kg-K)'] / (df['Density (kg/m³)'] * df['Thermal Conductivity (W/m-K)'])

# === Step 4: Detect material type ===
def detect_type(cat):
    if isinstance(cat, str) and any(word in cat.lower() for word in ['silicon', 'ceramic', 'oxide']):
        return 'Silicon-Based'
    else:
        return 'Carbon-Based'

df['Material_Type'] = df['Category'].apply(detect_type)

# === Step 5: Save feature-engineered dataset ===
df.to_csv("thermal_features.csv", index=False)
print("✅ Feature-engineered file saved as thermal_features.csv")

# === Step 6: Quick preview ===
print(df[['Material', 'Material_Type', 'TSEI']].head())

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import filedialog

# === Step 1: Select feature-engineered dataset ===
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename(title="📂 Select 'thermal_features.csv' File",
                                       filetypes=[("CSV Files", "*.csv")])
print(f"✅ File selected: {file_path}")

# === Step 2: Load data ===
df = pd.read_csv(file_path)

# === Step 3: Correlation matrix ===
plt.figure(figsize=(8, 6))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("🔹 Correlation Matrix of Thermal Properties")
plt.show()

# === Step 4: Distribution of TSEI ===
plt.figure(figsize=(7, 5))
sns.histplot(df['TSEI'], bins=10, kde=True, color='skyblue')
plt.title("📈 Distribution of Thermal Shielding Efficiency Index (TSEI)")
plt.xlabel("TSEI")
plt.ylabel("Frequency")
plt.show()

# === Step 5: Boxplot - Compare by Material Type ===
plt.figure(figsize=(6, 4))
sns.boxplot(x='Material_Type', y='TSEI', data=df, palette="Set2")
plt.title("🧩 TSEI Comparison: Silicon vs Carbon-Based Composites")
plt.show()

# === Step 6: Pairplot ===
sns.pairplot(df, vars=['Density (kg/m³)', 'Thermal Conductivity (W/m-K)',
                       'Specific Heat (J/kg-K)', 'TSEI'],
             hue='Material_Type', palette='husl')
plt.suptitle("Pairplot of Key Thermal Parameters", y=1.02)
plt.show()


In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

# === Step 1: File upload ===
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename(title="📂 Select 'thermal_features.csv' file",
                                       filetypes=[("CSV files", "*.csv")])
print(f"✅ File selected: {file_path}")

# === Step 2: Load dataset ===
df = pd.read_csv(file_path)

# === Step 3: Encode categorical variables ===
le = LabelEncoder()
df['Material_Type_Enc'] = le.fit_transform(df['Material_Type'])

# === Step 4: Define features & target ===
features = ['Density (kg/m³)', 'Thermal Conductivity (W/m-K)',
            'Specific Heat (J/kg-K)', 'TSEI', 'Material_Type_Enc']

target = 'Max Temp (K)'  # You can change to 'TSEI' if you want to predict efficiency instead
X = df[features]
y = df[target]

# === Step 5: Train-test split ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# === Step 6: Train Random Forest ===
model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# === Step 7: Predict and evaluate ===
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("\n🎯 Model Performance:")
print(f"R² Score: {r2:.3f}")
print(f"Mean Absolute Error: {mae:.3f}")

# === Step 8: Feature Importance ===
importance = pd.Series(model.feature_importances_, index=features)
plt.figure(figsize=(7,4))
sns.barplot(x=importance.values, y=importance.index, palette="coolwarm")
plt.title("🔥 Feature Importance (Predicting Max Temp)")
plt.xlabel("Importance Score")
plt.ylabel("Features")
plt.show()

# === Step 9: Save predictions ===
output = pd.DataFrame({"Actual": y_test, "Predicted": y_pred})
output.to_csv("predicted_results.csv", index=False)
print("✅ Predictions saved as 'predicted_results.csv'")
